In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/최종프로젝트'

- Naver Clova sentiment

In [3]:
with open(f'{path}/소스/ClientID_SEN.txt') as f:
  client_id = f.read()
len(client_id)

10

In [4]:
with open(f'{path}/소스/ClientSecret_SEN.txt') as f:
  client_secret = f.read()
len(client_secret)

40

In [75]:
data = {
  "content": "싸늘하다. 가슴에 비수가 날아와 꽂힌다."
}

In [81]:
import os
import sys
import requests

url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze" 

headers = {'X-NCP-APIGW-API-KEY-ID': client_id, 'X-NCP-APIGW-API-KEY': client_secret, 'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    # print(response.text)
    result = eval(response.text)
    confidence = result['document']['confidence']
    nega = confidence['negative']
    posi = confidence['positive']
    neut = confidence['neutral']
    score = [nega, posi, neut]
    sentiment = result['document']['sentiment']
    df = pd.DataFrame([[content, nega, posi, neut, sentiment]], columns = ['문장', '부정', '긍정', '중립', '결과'])
    print(df)

else:
    print("Error : " + response.text)

                                      문장       부정        긍정        중립        결과
0  {'content': '싸늘하다. 가슴에 비수가 날아와 꽂힌다.'}  99.3676  0.007764  0.624637  negative


In [82]:
import json
res = json.loads(response.text) # json에서 dictionary로 변환
res

{'document': {'confidence': {'negative': 99.3676,
   'neutral': 0.62463653,
   'positive': 0.0077638607},
  'sentiment': 'negative'},
 'sentences': [{'confidence': {'negative': 0.9961359,
    'neutral': 0.0036366575,
    'positive': 0.00022740211},
   'content': '싸늘하다.',
   'highlights': [{'length': 4, 'offset': 0}],
   'length': 5,
   'offset': 0,
   'sentiment': 'negative'},
  {'confidence': {'negative': 0.927976,
    'neutral': 0.07131973,
    'positive': 0.0007042612},
   'content': ' 가슴에 비수가 날아와 꽂힌다.',
   'highlights': [{'length': 15, 'offset': 1}],
   'length': 17,
   'offset': 5,
   'sentiment': 'negative'}]}

In [69]:
sentiment = res['document']['sentiment']
sentiment

'negative'

In [3]:
# 자료 가져오기
import pandas as pd
import numpy as np

contents = pd.read_csv('data.csv', skiprows=[0])
contents = np.array(contents).reshape(-1,1)
contents = pd.DataFrame(contents).dropna()
contents.columns = ['review'] 
contents

,review
0,스벅 토피넛라테🥜\n맛있네요😋\n\n#오늘의차☕️ #토피넛라떼 #스벅
1,#스타벅스#신메뉴#토피넛라떼#가격만비쌈#노맛#돈버림#절대먹지마세요#다버림#빵점#비추천
2,#스벅 #신메뉴 #커피\n기대했는데 그냥 그런듯..
3,#할로윈 #스벅 #soso #👀\nJMT 인듯 아닌듯\n맛이 있는것 같지도 없는것 ...
4,예쁜 비주얼 만큼이나 맛있는\n#아이스핑크캐모마일릴렉서\n\n#쟈샘일상#주부일상#일...
6,매일 아침 모닝 스벅.\n사실은 커피 셔틀🤣\n맛은 있는데...\n빨대가 불편..\...
7,#바닐라_미라_크림_프라푸치노 #스타벅스\n이미 품절이래요!\n어제 먹으러 갔는데 ...
8,#스타벅스 #토피넛콜드브루 #퍼플_추천\n라떼제외 커피를 마신다면 콜드브루보다는\n...


In [102]:
res_list = []
for text in contents['review']:
    in_text = {
        "content": text
        }
    response = requests.post(url, data=json.dumps(in_text), headers=headers)
    result = eval(response.text)
    confidence = result['document']['confidence']
    nega = confidence['negative']
    posi = confidence['positive']
    neut = confidence['neutral']
    score = [nega, posi, neut]
    sentiment = result['document']['sentiment']
    res_list.append([text, nega, posi, neut, sentiment])

df = pd.DataFrame(res_list, columns = ['문장', '부정', '긍정', '중립', '결과'])
df

,문장,부정,긍정,중립,결과
0,스벅 토피넛라테🥜\n맛있네요😋\n\n#오늘의차☕️ #토피넛라떼 #스벅,0.027415,59.995537,39.977050,positive
1,#스타벅스#신메뉴#토피넛라떼#가격만비쌈#노맛#돈버림#절대먹지마세요#다버림#빵점#비추천,89.945350,0.007334,10.047318,negative
2,#스벅 #신메뉴 #커피\n기대했는데 그냥 그런듯..,75.008200,0.004569,24.987232,negative
3,#할로윈 #스벅 #soso #👀\nJMT 인듯 아닌듯\n맛이 있는것 같지도 없는것 ...,28.606720,57.042830,14.350447,positive
4,예쁜 비주얼 만큼이나 맛있는\n#아이스핑크캐모마일릴렉서\n\n#쟈샘일상#주부일상#일...,0.029618,31.572800,68.397580,neutral
5,매일 아침 모닝 스벅.\n사실은 커피 셔틀🤣\n맛은 있는데...\n빨대가 불편..\...,75.008790,0.005077,24.986132,negative
6,#바닐라_미라_크림_프라푸치노 #스타벅스\n이미 품절이래요!\n어제 먹으러 갔는데 ...,66.697426,0.020436,33.282135,negative
7,#스타벅스 #토피넛콜드브루 #퍼플_추천\n라떼제외 커피를 마신다면 콜드브루보다는\n...,15.011949,60.003925,24.984127,positive


- 딥러닝 모델 이용을 위한 NLP

In [4]:
contents['cls'] = [1,-1,0,0,1,0,0,1] # 1: 긍정, 0: 중립, -1: 부정
contents

,review,cls
0,스벅 토피넛라테🥜\n맛있네요😋\n\n#오늘의차☕️ #토피넛라떼 #스벅,1
1,#스타벅스#신메뉴#토피넛라떼#가격만비쌈#노맛#돈버림#절대먹지마세요#다버림#빵점#비추천,-1
2,#스벅 #신메뉴 #커피\n기대했는데 그냥 그런듯..,0
3,#할로윈 #스벅 #soso #👀\nJMT 인듯 아닌듯\n맛이 있는것 같지도 없는것 ...,0
4,예쁜 비주얼 만큼이나 맛있는\n#아이스핑크캐모마일릴렉서\n\n#쟈샘일상#주부일상#일...,1
6,매일 아침 모닝 스벅.\n사실은 커피 셔틀🤣\n맛은 있는데...\n빨대가 불편..\...,0
7,#바닐라_미라_크림_프라푸치노 #스타벅스\n이미 품절이래요!\n어제 먹으러 갔는데 ...,0
8,#스타벅스 #토피넛콜드브루 #퍼플_추천\n라떼제외 커피를 마신다면 콜드브루보다는\n...,1


In [5]:
contents['review'] = contents['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9 ]", " ")

In [6]:
contents

,review,cls
0,스벅 토피넛라테 맛있네요 오늘의차 토피넛라떼 스벅,1
1,스타벅스 신메뉴 토피넛라떼 가격만비쌈 노맛 돈버림 절대먹지마세요 다버림 빵점 비추천,-1
2,스벅 신메뉴 커피 기대했는데 그냥 그런듯,0
3,할로윈 스벅 soso JMT 인듯 아닌듯 맛이 있는것 같지도 없는것 같지...,0
4,예쁜 비주얼 만큼이나 맛있는 아이스핑크캐모마일릴렉서 쟈샘일상 주부일상 일상 데...,1
6,매일 아침 모닝 스벅 사실은 커피 셔틀 맛은 있는데 빨대가 불편 ...,0
7,바닐라 미라 크림 프라푸치노 스타벅스 이미 품절이래요 어제 먹으러 갔는데 줄이...,0
8,스타벅스 토피넛콜드브루 퍼플 추천 라떼제외 커피를 마신다면 콜드브루보다는 아아...,1


In [7]:
!pip install Konlpy > /dev/null

In [8]:
df = pd.read_csv('/content/drive/MyDrive/project/기타참고자료/한국어불용어100.txt', sep='\s+', header=None)
# 품사를 무시하고 중복 배제
stopwords = set(df[0])

In [9]:
from konlpy.tag import Okt
okt = Okt()
morphs_str = []

for review in contents['review']:
    morphs = okt.morphs(review.strip(), stem=True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    morphs_str.append(temp_str)

contents['nlp'] = morphs_str
contents

,review,cls,nlp
0,스벅 토피넛라테 맛있네요 오늘의차 토피넛라떼 스벅,1,스벅 토피 넛 라테 맛있다 오늘 의 차 토피 넛 라떼 스벅
1,스타벅스 신메뉴 토피넛라떼 가격만비쌈 노맛 돈버림 절대먹지마세요 다버림 빵점 비추천,-1,스타벅스 신 메뉴 토피 넛 라떼 가격 만 비싸다 노맛 돈 버리다 절대 먹다 말다 다...
2,스벅 신메뉴 커피 기대했는데 그냥 그런듯,0,스벅 신 메뉴 커피 기대하다 그냥 그렇다
3,할로윈 스벅 soso JMT 인듯 아닌듯 맛이 있는것 같지도 없는것 같지...,0,할로윈 스벅 soso JMT 이다 아니다 맛 있다 같다 없다 같다 않다 신기하다 맛...
4,예쁜 비주얼 만큼이나 맛있는 아이스핑크캐모마일릴렉서 쟈샘일상 주부일상 일상 데...,1,예쁘다 비주 얼 만큼이나 맛있다 아이스 핑크 캐모마일 릴렉 서 쟈샘 일상 주부 일상...
6,매일 아침 모닝 스벅 사실은 커피 셔틀 맛은 있는데 빨대가 불편 ...,0,매일 아침 모닝 스벅 은 커피 셔틀 맛 은 있다 빨대 불편 핑크 캐모마일 릴렉 서 ...
7,바닐라 미라 크림 프라푸치노 스타벅스 이미 품절이래요 어제 먹으러 갔는데 줄이...,0,바닐라 미라 크림 프라푸치노 스타벅스 이미 품절 이래 요 어제 먹다 가다 줄이다 엄...
8,스타벅스 토피넛콜드브루 퍼플 추천 라떼제외 커피를 마신다면 콜드브루보다는 아아...,1,스타벅스 토피 넛 콜드 브루 퍼플 추천 라떼 제외 커피 를 말다 콜드 브루 보다는 ...
